In [2]:
pip install tinymlgen

  Using cached tinymlgen-0.2.tar.gz (1.5 kB)
  Using cached hexdump-3.3.zip (12 kB)
  Created wheel for tinymlgen: filename=tinymlgen-0.2-py3-none-any.whl size=2243 sha256=50371cccd46983d0a21b4308bce3b1d5bc81d53a1a49219ad4f50eed15b09bd3
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\7a\a1\f5\2884c682bcacd0a96875431737d5c792e8dd0a1e4e163512be
  Created wheel for hexdump: filename=hexdump-3.3-py3-none-any.whl size=8910 sha256=9a1eeec3869865a00ca18dc2368fb0f1d4ba2bbc68e02e2b1f19a43de3b076e1
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\7f\b7\72\1cc327e831ffef71bca1b0ca5e40b68471875f740ec9270c0f
Successfully built tinymlgen hexdump
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from sklearn.datasets import load_digits
import tensorflow as tf
from tensorflow.keras import layers
from tinymlgen import port

In [4]:
def get_data():
    np.random.seed(1337)
    x_values, y_values = load_digits(return_X_y=True)
    x_values /= x_values.max()
    # reshape to (8 x 8 x 1)
    x_values = x_values.reshape((len(x_values), 8, 8, 1))

    # split into train, validation, test
    TRAIN_SPLIT = int(0.6 * len(x_values))
    TEST_SPLIT = int(0.2 * len(x_values) + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

    return x_train, x_test, x_validate, y_train, y_test, y_validate

In [5]:
def get_model():
    x_train, x_test, x_validate, y_train, y_test, y_validate = get_data()

    # create a CNN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(8, 8, 1)))
    # model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(len(np.unique(y_train))))

    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=50, batch_size=16,
                        validation_data=(x_validate, y_validate))
    return model, x_test, y_test

In [6]:
def test_model(model, x_test, y_test):
    x_test = (x_test / x_test.max()).reshape((len(x_test), 8, 8, 1))
    y_pred = model.predict(x_test).argmax(axis=1)

    print('ACCURACY', (y_pred == y_test).sum() / len(y_test))

In [7]:
if __name__ == '__main__':
    model, x_test, y_test = get_model()
    test_model(model, x_test, y_test)
    c_code = port(model, variable_name='digits_model', pretty_print=True)
    print(c_code)

Epoch 1/50
68/68 [==============================] - 1s 6ms/step - loss: 2.0243 - accuracy: 0.4508 - val_loss: 1.6528 - val_accuracy: 0.7333
Epoch 2/50
68/68 [==============================] - 0s 3ms/step - loss: 1.1794 - accuracy: 0.8340 - val_loss: 0.9680 - val_accuracy: 0.7972
Epoch 3/50
68/68 [==============================] - 0s 2ms/step - loss: 0.6278 - accuracy: 0.8859 - val_loss: 0.7096 - val_accuracy: 0.8250
Epoch 4/50
68/68 [==============================] - 0s 2ms/step - loss: 0.4067 - accuracy: 0.9221 - val_loss: 0.6119 - val_accuracy: 0.8500
Epoch 5/50
68/68 [==============================] - 0s 2ms/step - loss: 0.3030 - accuracy: 0.9425 - val_loss: 0.5770 - val_accuracy: 0.8389
Epoch 6/50
68/68 [==============================] - 0s 2ms/step - loss: 0.2463 - accuracy: 0.9416 - val_loss: 0.5395 - val_accuracy: 0.8611
Epoch 7/50
68/68 [==============================] - 0s 2ms/step - loss: 0.2091 - accuracy: 0.9481 - val_loss: 0.5314 - val_accuracy: 0.8667
Epoch 8/50
68/68 [==

ACCURACY 0.9526462395543176


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp_7bu6kl9\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp_7bu6kl9\assets



#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

const unsigned char digits_model[] DATA_ALIGN_ATTRIBUTE = {
	0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00, 
	0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 
	0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 
	0x1c, 0x00, 0x00, 0x00, 0x88, 0x00, 0x00, 0x00, 0xe0, 0x00, 0x00, 0x00, 
	0x98, 0x0e, 0x00, 0x00, 0xa8, 0x0e, 0x00, 0x00, 0xe0, 0x13, 0x00, 0x00, 
	0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 
	0xca, 0xf0, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 
	0x38, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76, 
	0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x7